In [6]:
import pandas as pd
from typing import Tuple
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

class DataPrep:
  def __init__(self, data: pd.DataFrame) -> None:
    """Inicializa a classe DataPrep com a base de dados do Titanic."""
    self.data= data

  def remover_variaveis(self) -> None:
    """Remove as variáveis que não serão utilizadas pelo modelo."""
    colunas_para_remover= [
        "PassengerId",
        "Name",
        "Ticket",
        "Cabin", # Variável com muitos dados faltantes
        "Embarked", # Substituídas por variáveis dummies
        "SibSp", "Parch" # Foram combinadas em uma nova variável
    ]
    self.data.drop(columns= colunas_para_remover, inplace= True)

  def tratar_nulos(self) -> None:
    """Faz o tratamento das variáveis nulas, imputando o valor adequado."""
    self.data["Age"] = self.data.groupby(["Pclass", "Sex"], group_keys= False)["Age"].apply(lambda x: x.fillna(x.median()))
    self.data["Embarked"]= self.data["Embarked"].fillna("S")

  def tratar_variaveis_categoricas(self) -> None:
    """Faz o tratamento das variáveis categóricas."""
    sexo= {"male": 0, "female": 1} # Label Encoding
    self.data["Sex"]= self.data["Sex"].map(sexo)

    embarked_dummies= pd.get_dummies(self.data["Embarked"])
    self.data= pd.concat([self.data, embarked_dummies], axis= 1)

  def dimensionar_dados(self) -> None:
    variaveis= self.data.drop(columns= "Survived")
    var_cols= variaveis.columns
    resposta= self.data["Survived"]

    scaler= MinMaxScaler()
    variaveis= scaler.fit_transform(variaveis)
    variaveis= pd.DataFrame(variaveis, columns= var_cols)
    self.data= pd.concat([variaveis, resposta], axis= 1)

  def criar_variaveis(self) -> None:
    self.data["FamilySize"]= self.data["SibSp"] + self.data["Parch"] + 1

  def separa_treino_teste(self) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Separa a base de dados entre conjunto de treinamento e teste."""
    treino, teste= train_test_split(self.data, test_size= 0.3, random_state= 2021)
    return treino, teste

  def preparar_dados(self) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Executa todas as etapas de transformação de dados."""
    self.tratar_nulos()
    self.tratar_variaveis_categoricas()
    self.criar_variaveis()
    self.remover_variaveis()
    self.dimensionar_dados()
    treino, teste= self.separa_treino_teste()
    return treino, teste